Test of PDF Reader into String (one demo file only)

In [1]:
#Drawn from https://www.geeksforgeeks.org/working-with-pdf-files-in-python/ via Sophia

import PyPDF2
#pdfFileObj = open('/Volumes/GoogleDrive-105310167471895906199/My Drive/1 - Coursework/10 - 2021-22/UP 229/WhoPlansForSchoolTrips/TestPDFs/1-04-11CSMinutes.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#print(pdfReader.numPages) #prints number of pages in the 
#CAN READ ALL PAGES _FIX
#pageObj = pdfReader.getPage(0)   #creates a page object, but I have no idea why?
#print(pageObj.extractText())   #shows text -- would comment this out when applying to whole pot
#pdfFileObj.close()  #closes the object, but I'm not 100% sure why this is needed beyond it's how the code reads the PDF and then needs to close it?

In [2]:
#this is me trying to purely read the pdf file - works but not readable
import urllib3
http = urllib3.PoolManager()
r = http.request('GET', 'http://laschoolboard.org/sites/default/files/1-04-11CSMinutes.pdf')

In [1]:
#print(r.data)

In [5]:
import urllib.request
#borrowed this from here https://www.codegrepper.com/code-examples/python/use+urllib3+in+python+3+to+get+pdf+file+from+url
#download file into 'files' folder

#read said downloaded file THIS IS OBSOLETE - PDF MINER WORKS MUCH BETTER
def read_download(name):
    pdfFileObj = open('files/{}.pdf'.format(name), 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0)  
    text = pageObj.extractText()   
    pdfFileObj.close()
    return text   
#apply this to dataframe
#inputs are two lists- as in the dataframe 
#only go into pdf reading if the length of list of names/links is greater than 0 
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open("files/" + filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()
def application (pdfnames, pdfurls):
    texts = []
    if len(pdfnames) > 0:
        for pdfname, pdfurl in zip(pdfnames, pdfurls):
            download_file(pdfurl, pdfname)
            text = extract_text("files/{}.pdf".format(pdfname))
            text_wordsonly = re.sub(r"[^A-z\s]", "", text)
            text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
            texts.append(text_wordsonly)
    return texts

In [6]:
pdf_path = "http://laschoolboard.org/sites/default/files/1-04-11CSAgenda.pdf"
name = 'r'

download_file(pdf_path, name)
from pdfminer.high_level import extract_text
import re
text = extract_text("files/{}.pdf".format(name))
text_wordsonly = re.sub(r"[^A-z\s]", "", text)
text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
print(text_wordsonly)

BOARD OF EDUCATION OF THE CITY OF LOS ANGELES Governing Board of the Los Angeles Unified School District SPECIAL MEETING ORDER OF BUSINESS South Beaudry Avenue Board Room am Tuesday January A Roll Call of Members B Public Comment on Items to be Discussed at This Meeting The public can address the Board at the commencement of the meeting in the Board Room on any item that is described in this notice At the conclusion of the closed session portion of the meeting announcements required regarding actions taken by the Board of Education will be made in the Board Room C Closed Session Items Purpose and Authority Conference with Legal Counsel Existing Litigation Government Code Section a Los Angeles Unified School District v Dolly Harris et al Los Angeles Superior Court Case No BC Conference with Labor Negotiator Government Code Negotiator Superintendent Cortines Employee Organizations Associated Administrators of Los Angeles California School Employees Association Los Angeles County Building

In [52]:
import urllib3
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from io import BytesIO

def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    f = urllib.request.urlopen(url).read()
    fp = BytesIO(f)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str
x = pdf_from_url_to_txt(re.sub(r" ", "%20", df.files[6][0]))
text_wordsonly = re.sub(r"[^A-z\s]", "", x)
text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)

def application (pdfnames, pdfurls):
    texts = []
    if len(pdfnames) > 0:
        for pdfname, pdfurl in zip(pdfnames, pdfurls):
            text = pdf_from_url_to_txt(re.sub(r" ", "%20", pdfurl))
            text_wordsonly = re.sub(r"[^A-z\s]", "", text)
            text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
            texts.append(text_wordsonly)
    return texts

In [53]:
df['text'] =df.apply(lambda x: application(x['names'], x['files']), axis = 1)

KeyError: 10

# Summary

-Need to update application function to work for full dataframe.

-Figure out how to read charters

In [9]:
import pandas as pd
df = pd.read_pickle('links12.pandas')
pdf_path = df.files[3][0]
name = 'trial'
download_file(pdf_path, name)
text = extract_text("files/{}.pdf".format(name))
text_wordsonly = re.sub(r"[^A-z\s]", "", text)
text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
#print(pdf_path)
print(text_wordsonly)
#charters don't work
#minutes usually work
#application(df.files,df.names)

BOARD OF EDUCATION OF THE CITY OF LOS ANGELES Governing Board of the Los Angeles Unified School District REGULAR MEETING MINUTES South Beaudry Avenue Board Room am Tuesday January The Los Angeles Board of Education acting as the Governing Board of the Los Angeles Unified School District met in special session on Tuesday January at the Administrative Offices Board Room South Beaudry Avenue Los Angeles California Board President Mnica Garca called the meeting to order at am The following Board Members were present Ms Tamar Galatzan Ms Marguerite P LaMotte Ms Nury Martinez Dr Richard Vladovic and President Garca Mr Steven Zimmer and Mr Bennett Kayser arrived at am Closed Session Items Purpose and Authority Mr Jefferson Crain Executive Officer of the Board announced that the following items would be discussed in closed session Personnel Government Code Section Public Employment Associate General Counsel II Employee DisciplineDismissalRelease Employee Evaluation Superintendent Inspector Gen

In [12]:
def application (pdfnames, pdfurls):
    texts = []
    if len(pdfnames) > 0:
        for pdfname, pdfurl in zip(pdfnames, pdfurls):
            download_file(pdfurl, pdfname)
            text = extract_text("files/{}.pdf".format(pdfname))
            text_wordsonly = re.sub(r"[^A-z\s]", "", text)
            text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
            texts.append(text_wordsonly)
    return texts

print(application(df.names,df.files))

AttributeError: 'list' object has no attribute 'timeout'